In [10]:
#% matplotlib inline
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import glob
from matplotlib.ticker import MultipleLocator, FormatStrFormatter

# read in meta data file for MWSS
filename='meta data for MWSS.csv'
df=pd.read_csv(filename, header=0, sep=',')

# dataframe of only wells with gamma and SP
df2=df.loc[(df.glogs.str.contains('GR') & (df.glogs.str.contains('SP')))]

# narrow down df2 by depths
df3=df2.loc[(df2.start_depth <= 50) & (df2.stop_depth <= 650)]
print 'Number of wells:', len(df3)

df3.head()

Number of wells: 4


,well_num,wellID,latitude,longitude,start_depth,stop_depth,field,glogs
415,40295495700,17S0290,35.140679,-119.529631,30.0,638.0,MIDWAY-SUNSET,"['CALI', 'DPHI', 'DRES', 'GR', 'NPHI', 'SP', '..."
416,40296101900,17S0291,35.140345,-119.529130,48.0,620.0,MIDWAY-SUNSET,"['DRES', 'GR', 'SP', 'SRES']"
423,40296102300,17S0307,35.140710,-119.528511,45.0,573.0,MIDWAY-SUNSET,"['DRES', 'GR', 'SP']"
428,40296123400,17S0320,35.142030,-119.530198,34.0,616.0,MIDWAY-SUNSET,"['DRES', 'GR', 'SP', 'SRES']"


In [11]:
# plotting fuction

def gamma_SP_plot(filename):
    
    # make a dataframe
    f=open(filename)
    lines=f.readlines()
    f.close()  
    counter=0 
    for line in lines: 
        l=line.strip().split()
        if l[0]=='~A':
            counter
            break
        else:
            counter = counter + 1
    line = lines[counter].strip().split()
    names = line[1:]
    APInum = lines[20].strip().split()  
    df = pd.read_table(filename, skiprows = counter+1, names=names, sep = '\s+')   
    df[df < -990.00] = None
    
    # skips shallow depths
    #df = df.loc[(df.DEPTH >= min(df.DEPTH) + 150)]
    
    # plot gamma and SP
    fig, ax1 = plt.subplots(figsize=(17,6))
    plt.grid()
    plt.title('MWSS gamma and SP\nAPI:0{}'.format(APInum[2][3:-3]), x=0.5, y=1.07, fontsize=17)
    
    ax2 = ax1.twinx()
    ax1.plot(df.DEPTH, df.SP, 'b')
    ax2.plot(df.DEPTH, df.GR, 'r')
    
    ax1.set_xlabel('Depth (ft)')
    ax1.set_ylabel('SP (mV)', color='b', size=13)
    ax2.set_ylabel('Gamma (GAPI)', color='r', size=13)
    
    # more ticks and grids
    ax1.xaxis.set_major_locator(MultipleLocator(50.0))
    ax1.xaxis.set_major_formatter(FormatStrFormatter('%i'))
    
    plt.tight_layout()
    plt.show()

In [18]:
# try it out on one
#gamma_SP_plot('17S0778_OH_MWSS.las')

In [12]:
# grab all well files in folder
files=glob.glob('*.las')

# well IDs for the ones we want
wells=np.array(df3.wellID)

# these loops find if the wellID is in the filename and if so, runs the plotting function 
try:
    for g in files:
        for gg in wells:
            if gg in g:
                gamma_SP_plot(g)
except AttributeError:
    print g